In [ ]:
import sys
import logging
import tomli
import pandas as pd

import pytanis
from pytanis import GSheetsClient, PretalxClient
from pytanis.google import Scope, gsheet_rows_for_fmt, worksheet_range
from pytanis.pretalx import subs_as_df, reviews_as_df, speakers_as_df
from pytanis.utils import implode


In [ ]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

In [ ]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)


## Retrieve the talks

In [ ]:
pretalx_client = PretalxClient(blocking=True)
talks_count, talks = pretalx_client.submissions(cfg['event_name'], params={"questions": "all", "state": ["confirmed"]})
talks = list(talks)

In [ ]:
def to_pdf(subs):
    rows = []
    for sub in subs:
        if not sub.slot:
            print("Not scheduled?")
            print(sub.speakers, sub.title, sub.code)
            continue
        row = {
            "Submission": sub.code,
            "Speaker": ", ".join([speaker.name for speaker in sub.speakers]),
            "Day": f"{sub.slot.start.day}",
            "Time": f"{sub.slot.start.time()}-{sub.slot.end.time()}",
            "Room": sub.slot.room.en,
            "Duration": sub.duration,
            "Track": sub.track.en if sub.track else None,
            "Title": sub.title,
            "URL": f"https://pretalx.com/pyconde-pydata-berlin-2023/talk/{sub.code}/"
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [ ]:
df = to_pdf(talks)

In [ ]:
rooms = list(df['Room'].unique())
rooms

In [ ]:
columns_list = ["Day", "Time", "Speaker", "Title", "Duration", "Submission", "URL"]

## Upload to GSheet

In [ ]:
gsheet_client = GSheetsClient(read_only=False)

In [ ]:
for room in rooms:
    gsheet_client.save_df_as_gsheet(df[df['Room']==room][columns_list].sort_values(by=['Day', 'Time']), cfg['schedule_spread_id'], room, create_ws=True)